VERSION 0.1 ON 30072022 - 23.52



I HAVE BEEN ALLOTTED 80 GRB FILES (NAMES STORED IN THE VARIABLE $folders$)

OUT OF THOSE 80 FILES, 22 ARE MASTERSAMPLE FILES SO THEY HAVE BEEN IGNORED AS PER INSTRUCTIONS

THE REMAINING 52 FILES ARE WITHOUT DUPLICATE BANDS IN THE SAME TIME AND/OR LIMITING MAGNITUDES

ANY POINTS SEEN OVERLAPPING ARE SIMPLY VERY CLOSE TO EACH OTHER

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import kaleido
from PyPDF2 import PdfFileMerger
import scipy
import matplotlib.pyplot as plt
#from sklearn.linear_model import LinearRegression

In [2]:
#FILE PATHS
#systempath = "/media/darkwake/VIB2/"
systempath = ''
inputfilepath = systempath + "task3/v_converted_filtered_files/with_outliers/"
output_file_path = systempath + "task3/v_converted_filtered_files/"
output_pdf_path = systempath + "task3/v_out_pdfs/"
#THE 80 FILES ALOTTED TO ME
folders = "140423A ,140428A ,140430A ,140506A ,140508A ,140509A ,140512A ,140515A ,140518A ,140606B ,140614A ,140620A ,140623A ,140629A ,140703A ,140710A ,140801A ,140808A ,140903A ,140907A ,141004A ,141028A ,141109A ,141121A ,141220A ,141221A ,141225A ,150101B ,150120B ,150206A ,150301B ,150314A ,150323A ,150403A ,150413A ,150423A ,150514A ,150518A ,150727A ,150818A ,150821A ,150910A ,150915A ,151021A ,151027A ,151027B ,151029A ,151031A ,151111A ,151112A ,151215A ,160117B ,160121A ,160131A ,160203A ,160227A ,160228A ,160314A ,160410A ,160425A ,160509A ,160623A ,160625B ,160629A ,160804A ,160821B ,161014A ,161017A ,161023A ,161108A ,161117A ,161129A ,161219B ,170113A ,170202A ,170214A ,170405A ,170428A ,170519A"

#22 MASTERSAMPLE FILES WITHIN MY ALOTTED FILES
masterfolders = "140423A,140430A,140801A,140907A,141121A,141221A,150323A,150413A,150910A,151027A,151029A,160131A,160227A,160314A,160425A,160804A,161014A,161023A,161219B,170113A,170405A, 170531B"
folders = folders.split(" ,")
masterfolders = masterfolders.split(",")


#ADDITIONAL PRECURSOR FILES
v_folders = ['121027A', '121201A', '121209A', '121211A', '121229A', '130408A', '130418A', '130427B', '130505A', '130511A', '130514A', '130518A', '130603B', '130610A', '130612A', '130701A', '130907A', '130925A', '131004A', '131011A', '131030A', '131103A', '131105A', '131108A', '131117A', '131231A', '140114A', '140213A', '140226A', '140301A', '140304A', '140311A', '140318A', '140419A']

#Biagio's Additional files - are merged in a separate .pdf
b_folders = ['970228A', '971214A', '980329A', '980425A', '980613A', '980703A', '990123A', '990712A', '991208A']
#REMOVING THE MASTERSAMPLE FILES FROM THE LIST OF FILES TO BE SCANNED
for i in masterfolders:
    if i in folders:
        folders.remove(i)
v_folders.extend(folders)

filenames = [i + "_converted_filtered_flux" for i in v_folders]
b_filenames = [i + "_converted_filtered_flux" for i in b_folders]

In [3]:
#FUNCTION TO CONCATANATE N DATA FRAMES INTO A SINGLE DATAFRAME AND RETURN IT
def concat(*args):
    return pd.concat([x for x in args if not x.empty])

In [4]:
#QUADRATIC EXPRESSION
def poly(x, a, b, c):
    return a * (x**2) + b * x + c

curvesl = LinearRegression()
#curvesl.fit(np.log10(light2.index.values).reshape(-1, 1), np.log10(light2.flux.values))
t2 = np.array([np.log10(light2.index.values),np.square(np.log10(light2.index.values))]).T
curvesl.fit(t2, np.log10(light2.flux.values))
m = curvesl.coef_
c = curvesl.intercept_
flux_pred = []
for i in light2.index.values:
    flux_pred.append(poly(np.log10(i), m[1], m[0], c))

In [ ]:
1 in range(-1, 3)

In [ ]:
filenames[30]

In [9]:
filenames[26]

'140114A_converted_filtered_flux'

In [10]:
light = pd.read_csv(inputfilepath + filenames[26]+".txt", engine="python", names= ["time_sec","flux","flux_err","band","source"],
         header=0, index_col=0)

In [11]:
np.log10(light.flux_err.values)

array([-14.86212306])

In [12]:
np.log10(light.flux_err.values)

array([-14.86212306])

In [18]:
def fit(light):
    if len(light) in range(1, 7):
        return 0
    if len(light) == 0:
        return 1
        
    light2 = light.sort_index()
    curves=scipy.optimize.curve_fit(poly, np.log10(light2.index.values), np.log10(light2.flux.values) )[0]
    flux_pred = []
    for i in light2.index.values:
        flux_pred.append(poly(np.log10(i), curves[0], curves[1], curves[2]))

    return flux_pred

In [14]:
def poly2(x, a, b):
    return 2 * a * x + b

In [63]:
def rem_outliers(light,flux_pred):
    
    if isinstance(light, int) or isinstance(light, float):
        return 1
    
    if len(light) in range(1, 7):
        return 0
    if len(light) == 0:
        return 1
    
    light2 = light.sort_index()
    for i in range(len(light2.index.values)):
        if abs(np.log10(flux_pred[i]) - np.log10(light.flux.values[i])) > 0.3:
            light2.flux.values[i] = None

    light2.dropna(inplace=True)  
    
    return light2

with_outliers = "with_outliers/"
def plot(light, flux_pred, filename):
    filename2 = filename.replace("_flux", "_curve_fitted_flux_with_outliers")
    
    if len(light) == 0:
        fig = px.scatter(title=filename + ' NULL AFTER FILTERING')
        fig.write_image(output_pdf_path + with_outliers+ filename2+".pdf")
        return 1

    if flux_pred == 0:
        fig = px.scatter(
                data_frame=light,
                x=np.log10(light.index),
                y=np.log10(light.flux.values),
                error_y= light.flux_err.values / (light.flux.values * np.log(10)),
                color=light['band'].values,
                hover_data=['source'],
                title=filename + " MORE DATA NEEDED"
            )
        fig.update_layout(
                xaxis_title='log Time (s)',
                yaxis_title='log Flux (erg cm-2 s-1)'
            )
        fig.write_image(output_pdf_path + with_outliers+ filename2+".pdf")
        return 0

    

    light2 = light.sort_index()
    fig = px.scatter(
                data_frame=light2,
                x=np.log10(light2.index),
                y=np.log10(light2['flux'].values),
                error_y= light2['flux_err'].values / (light2['flux'].values * np.log(10)),
                color=light2['band'].values,
                hover_data=['source'],
                title=filename,width=1200, height=750
            )
    fig.add_trace(go.Scatter(x=np.log10(light2.index.values), y=flux_pred,mode="lines", line = go.scatter.Line(color='gray'), showlegend=False))
    fig.update_layout(
                xaxis_title='log Time (s)',
                yaxis_title='log Flux (erg cm-2 s-1)'
            )
    
    #fig.show()
    fig.write_image(output_pdf_path + with_outliers+ filename2+".pdf")

flux_pred = fit(light)
plot(light, fit(light), filenames[16])

In [64]:
no_outliers = "no_outliers/"
def plot(light, flux_pred, filename):
    filename2 = filename.replace("_flux", "_curve_fitted_flux_no_outliers")
    
    if isinstance(light, int) or isinstance(light, float):
        fig = px.scatter(title=filename + ' NULL AFTER FILTERING')
        fig.write_image(output_pdf_path + no_outliers+ filename2+".pdf")
        return 1

    if flux_pred == 0:
        fig = px.scatter(
                data_frame=light,
                x=np.log10(light.index),
                y=np.log10(light.flux.values),
                error_y= light.flux_err.values / (light.flux.values * np.log(10)),
                color=light['band'].values,
                hover_data=['source'],
                title=filename + " MORE DATA NEEDED"
            )
        fig.update_layout(
                xaxis_title='log Time (s)',
                yaxis_title='log Flux (erg cm-2 s-1)'
            )
        fig.write_image(output_pdf_path + no_outliers + filename2+".pdf")
        return 0

    

    light2 = light.sort_index()
    fig = px.scatter(
                data_frame=light2,
                x=np.log10(light2.index),
                y=np.log10(light2['flux'].values),
                error_y= light2['flux_err'].values / (light2['flux'].values * np.log(10)),
                color=light2['band'].values,
                hover_data=['source'],
                title=filename,width=1200, height=750
            )
    fig.add_trace(go.Scatter(x=np.log10(light2.index.values), y=flux_pred,mode="lines", line = go.scatter.Line(color='gray'), showlegend=False))
    fig.update_layout(
                xaxis_title='log Time (s)',
                yaxis_title='log Flux (erg cm-2 s-1)'
            )
    
    #fig.show()
    fig.write_image(output_pdf_path + no_outliers+ filename2+".pdf")

file = filenames[1]
light = pd.read_csv(inputfilepath + file+".txt", engine="python", names= ["time_sec","flux","flux_err","band","source"],
         header=0, index_col=0)
light

plot(light, fit(light), file)

light.to_csv("task3/v_converted_filtered_files/"+file + "no_outliers.txt")

In [65]:
i=0
for file in filenames:
    i+=1
    light2 = pd.read_csv(inputfilepath + file+".txt", engine="python", names= ["time_sec","flux","flux_err","band","source"],
         header=0, index_col=0)
    fl = fit(light2)
    l2 = rem_outliers(light2,fl)
    plot(l2, fl, file)

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10


In [61]:
l2 = rem_outliers(light2,fl)

In [62]:
l2

,flux,flux_err,band,source
time_sec,,,,


In [68]:
i=0
for file in b_filenames:
     i+=1
     light = pd.read_csv(inputfilepath + file+".txt", engine="python", names= ["time_sec","flux","flux_err","band","source"],
         header=0, index_col=0)
     fl = fit(light)
     l2 = rem_outliers(light,fl)
     plot(l2, fl, file)

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10

/tmp/ipykernel_21485/917039953.py:13: RuntimeWarning:

invalid value encountered in log10



THE FOLLOWING CELL MERGES ALL THE OUTPUT PDF FILES INTO A SINGLE PDF FILE: $ALL$ _ $LCS$ _ $FOR$ _ $500$ _ $filtered$ _ $merged.pdf$

In [70]:
pdfs = [output_pdf_path + no_outliers + file.replace( "_flux","_curve_fitted_flux_no_outliers.pdf")  for file in filenames]
merger = PdfFileMerger()

for pdf in pdfs:
    merger.append(pdf)
    
merger.write(output_pdf_path + no_outliers + "ALL_LCS_FOR_500_filtered_no_outliers_merged_v.pdf")
#merger.write(output_pdf_path + "merged_not_done.pdf")
merger.close()

In [71]:
pdfs = [output_pdf_path + no_outliers + file.replace( "_flux","_curve_fitted_flux_no_outliers.pdf")  for file in b_filenames]
merger = PdfFileMerger()

for pdf in pdfs:
    merger.append(pdf)
    
merger.write(output_pdf_path + no_outliers + "ALL_LCS_FOR_500_filtered_no_outliers_merged_b.pdf")
#merger.write(output_pdf_path + "merged_not_done.pdf")
merger.close()